## Used Car Price Prediction

### 1) Problem Statement
        - This Dataset comprises used cars sold on cardekho.com in India as well as important features of these cars.
        - If user can predict the price of the car based on the input features.
        - Prediction results can be used to give a new seller the price suggestion based on market condition.

### 2) Data Collection
    
    - The Dataset is collected from scrapping from cardekho website
    - The data consist of 13 columns and 15411 rows

In [1]:
## Importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
dataset = pd.read_csv('cardekho_imputated.csv', index_col=0)

In [6]:
dataset.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000
